# CDAE

- [Collaborative Denoising Auto-Encoders for Top-N Recommender Systems](https://alicezheng.org/papers/wsdm16-cdae.pdf)

## Experiment

- K-fold 사용
    - 귀찮아서 각 모델을 따로 저장하지 않고 연속적으로 학습시킴

In [97]:
import glob
import numpy as np
import pandas as pd
from typing import Callable, Tuple

import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
plt.style.use('bmh')

from sklearn.model_selection import train_test_split, KFold

import tensorflow as tf
import tensorflow.keras.backend as K
from tensorflow.keras.layers import Input, Reshape, Dense, Dropout, Embedding, Flatten, concatenate
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from tensorflow.keras.optimizers import SGD, RMSprop, Adam
from tensorflow.keras.optimizers.schedules import ExponentialDecay
from tensorflow.keras.initializers import TruncatedNormal, RandomNormal
from tensorflow.keras.regularizers import l2

from tensorflow.keras.utils import get_file
import zipfile

In [98]:
def load_data(data_size : str) -> pd.DataFrame:
    ''' load Movie Lens data '''
    if data_size == '100k':
        file = get_file('ml-100k.zip', 'http://files.grouplens.org/datasets/movielens/ml-100k.zip')
        file_name = 'ml-100k/*'
    elif data_size == '1m':
        file = get_file('ml-1m.zip', 'http://files.grouplens.org/datasets/movielens/ml-1m.zip')
        file_name = 'ml-1m/ratings.dat'
    elif data_size == '10m':
        file = get_file('ml-10m.zip', 'http://files.grouplens.org/datasets/movielens/ml-10m.zip')
        file_name = 'ml-10M100K/ratings.dat'
    elif data_size == '20m':
        file = get_file('ml-20m.zip', 'http://files.grouplens.org/datasets/movielens/ml-20m.zip')
        file_name = 'ml-20m/ratings.csv'
    zip_ref = zipfile.ZipFile(file, 'r')
    zip_ref.extractall()

    col_names = ['userId', 'movieId', 'rating', 'timestamp']
    ratings = pd.read_csv(file_name, sep = '|', delimiter = '::', names = col_names, engine = 'python')
    print(ratings.shape)
    return ratings

In [99]:
ratings = load_data('1m')
ratings.head()

(1000209, 4)


,userId,movieId,rating,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


## 1. Data Preprocessing

In [100]:
def reduce_sparsity(df: pd.DataFrame, threshold = 3.5) -> pd.DataFrame:
    df = df[df.rating > threshold]
    more_than_4 = df.groupby('userId')['movieId'].nunique()
    more_than_4 = more_than_4.index[more_than_4 > 4]
    df = df[df.userId.isin(more_than_4)]
    return df.reset_index(drop = True)

In [101]:
idx_user_map = ratings.userId.unique()
user_idx_map = {e: i for i, e in enumerate(idx_user_map)}
n_user = idx_user_map.shape[0]
print(f'# of user = {n_user}')

idx_item_map = ratings.movieId.unique()
item_idx_map = {e: i for i, e in enumerate(idx_item_map)}
n_item = idx_item_map.shape[0]
print(f'# of item = {n_item}')

# of user = 6040
# of item = 3706


In [102]:
def Id2idx(df: pd.DataFrame) -> pd.DataFrame:
    return df.assign(userId = lambda x: x.userId.map(user_idx_map), 
                     movieId = lambda x: x.movieId.map(item_idx_map))

def idx2Id(df: pd.DataFrame) -> pd.DataFrame:
    return df.assign(userId = lambda x: x.userId.apply(lambda x: idx_user_map[x]), 
                     movieId = lambda x: x.movieId.apply(lambda x: idx_item_map[x]))

In [103]:
ratings = reduce_sparsity(ratings)
ratings = Id2idx(ratings)

print(ratings.shape)
ratings.head(5)

(575272, 4)


,userId,movieId,rating,timestamp
0,0,0,5,978300760
1,0,3,4,978300275
2,0,4,5,978824291
3,0,6,5,978302039
4,0,7,5,978300719


In [104]:
def data_split(df: pd.DataFrame, test_size = 0.2) -> pd.DataFrame:
    train, test = train_test_split(df, test_size = test_size, stratify = df.userId, random_state = 7777)
    train, test = map(lambda df: df.reset_index(drop = True), (train, test))
    return train, test

In [105]:
def make_interaction(df: pd.DataFrame) -> pd.DataFrame:
    return df.groupby('userId', as_index = False)[['movieId', 'rating']].agg(list)

In [106]:
def make_generator(df: pd.DataFrame,
                   n_item: int,
                   batch_size: int,
                   implicit = False,
                   threshold = 3.5,
                   user_specific = True,
                   corruption = 0.4,
                   negative_sampling = 5) -> Tuple[np.array, Tuple[np.array, np.array]]:

    n_row = df.index.size
    n_col = n_item

    Ids = np.arange(n_row)
    profile = df['movieId']
    rating = df['rating'] if not implicit else df['rating'].apply(lambda x: [1 if r > threshold else 0 for r in x])

    n_batch = int(np.ceil(n_row / batch_size))
    while True:
        np.random.shuffle(Ids)
        
        for batch_step in range(n_batch):
            lower = batch_step * batch_size
            upper = lower + batch_size
            
            batch_Id = Ids[lower: upper]
            batch = np.zeros(shape = (batch_Id.size, n_col))
            for i, idx in enumerate(batch_Id):
                pos = np.array(profile[idx])
                NS = len(pos) * negative_sampling
                neg = np.random.choice(np.delete(np.arange(n_col), pos), NS)
                p = np.random.uniform(0, 1, size = pos.shape)
                corrupt = pos[p < corruption]

                batch[i, pos] = rating[idx]
                batch[i, corrupt] = 0
                batch[i, neg] = np.finfo(float).eps
            
            if user_specific:
                yield (batch, batch_Id), batch
            else:
                yield batch, batch

In [107]:
def masked_mae(y_true: np.array, y_pred: np.array, masked_value = 0) -> tf.Tensor:
    mask_true = tf.cast(tf.not_equal(y_true, masked_value), dtype = 'float32')
    masked_ae = K.abs(mask_true * (y_true - y_pred))
    return K.sum(masked_ae, axis = -1) / K.sum(mask_true, axis = -1)

def masked_rmse(y_true: np.array, y_pred: np.array, masked_value = 0) -> tf.Tensor:
    mask_true = K.cast(K.not_equal(y_true, masked_value), dtype = 'float32')
    masked_se = K.square(mask_true * (y_true - y_pred))
    masked_mse = K.sum(masked_se, axis = -1) / K.sum(mask_true, axis = -1)
    return K.sqrt(masked_mse)
   
def masked_cross_entropy(y_true: np.array, y_pred: np.array, masked_value = 0) -> tf.Tensor:
    mask_true = K.cast(K.not_equal(y_true, masked_value), dtype = 'float32')
    ce = y_true * K.log(y_pred + K.epsilon()) + (1 - y_true) * K.log(1 - y_pred + K.epsilon())
    masked_ce = mask_true * ce
    return -K.sum(masked_ce, axis = -1) / K.sum(mask_true, axis = -1)

In [108]:
def CDAE(n_user: int,
         n_item: int,
         latent_dim = 50,
         activation = 'linear',
         optimizer = 'adam',
         learning_rate = 0.001,
         kernel_initializer = None,
         kernel_regularizer = None,
         user_specific = True) -> Callable:

    if not kernel_initializer:
        kernel_initializer = RandomNormal(mean = 0.0, stddev = 0.05, seed = None)
    if not kernel_regularizer:
        kernel_regularizer = l2(0.0001)
    
    inputs = x = Input(shape = (n_item, ), name = 'input')
    x = Dense(latent_dim, activation = activation, kernel_initializer = kernel_initializer, kernel_regularizer = kernel_regularizer, name = 'encoder')(x)

    if user_specific:
        users = u = Input(shape = (1, ), dtype = 'int32', name = 'user')
        u = Embedding(input_dim = n_user, output_dim = latent_dim, input_length = 1, embeddings_regularizer = kernel_regularizer)(u)
        u = Flatten()(u)
        x = concatenate([x, u], axis = 1)
    
    outputs = Dense(n_item, activation = 'sigmoid', kernel_initializer = kernel_initializer, kernel_regularizer = kernel_regularizer, name = 'decoder')(x)
    if user_specific:
        model = Model(inputs = [inputs, users], outputs = outputs)
    else:
        model = Model(inputs = inputs, outputs = outputs)

    model.compile(optimizer = optimizer, loss = masked_cross_entropy, metrics = [masked_cross_entropy, masked_rmse])
    return model

In [109]:
def make_predictor(df: pd.DataFrame, 
                   model: Callable, 
                   n_item: int, 
                   batch_size: int, 
                   implicit = True,
                   threshold = 3.5,
                   user_specific = True,
                   top_N = 20) -> float:

    n_row = df.index.size
    n_col = n_item

    Ids = np.arange(n_row)
    profile = df['movieId']
    rating = df['rating'] if not implicit else df['rating'].apply(lambda x: [1 if r > threshold else 0 for r in x])

    res = []
    steps = int(np.ceil(n_row / batch_size))
    for batch_step in range(steps):
        lower = batch_step * batch_size
        upper = lower + batch_size

        batch_Id = Ids[lower: upper]
        y_true = np.zeros(shape = (batch_Id.size, n_col))
        for i, idx in enumerate(batch_Id):
            y_true[i, profile[idx]] = rating[idx]

        if user_specific:
            y_pred = model.predict([y_true, batch_Id], verbose = 0)
        else:
            y_pred = model.predict(y_true, verbose = 0)
        
        pred = np.argsort(np.where(y_true == 1, 0, 1) * y_pred, axis = -1)[:, :-top_N-1:-1]
        res.append(pred)
    return np.concatenate(res)

In [110]:
def show_history(hist, loss: str, metric: str) -> None:
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize = (12, 4))

    ax1.plot(hist.history[f'{loss}'])
    ax1.plot(hist.history[f'val_{loss}'])
    ax1.set_title('Loss', fontsize = 20)
    ax1.set_ylabel('loss')
    ax1.set_xlabel('epoch')
    ax1.legend(['loss', 'val_loss'], loc = 'upper right')

    ax2.plot(hist.history[f'{metric}'])
    ax2.plot(hist.history[f'val_{metric}'])
    ax2.set_title('Metric', fontsize = 20)
    ax2.set_ylabel('metric')
    ax2.set_xlabel('epoch')
    ax2.legend(['masked_metric', 'val_masked_metric'], loc = 'upper right')
    plt.show()

## 2. DAE

In [111]:
optimizer = RMSprop(learning_rate = 0.001, decay = 1e-5)
model = CDAE(n_user, n_item, latent_dim = 512, optimizer = optimizer, user_specific = False)
model.summary()

Model: "functional_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input (InputLayer)           [(None, 3706)]            0         
_________________________________________________________________
encoder (Dense)              (None, 512)               1897984   
_________________________________________________________________
decoder (Dense)              (None, 3706)              1901178   
Total params: 3,799,162
Trainable params: 3,799,162
Non-trainable params: 0
_________________________________________________________________


In [112]:
train, test = data_split(ratings)
train, test = map(make_interaction, (train, test))

In [113]:
epochs = 100
batch_size = 128

n_splits = 5
folds = KFold(n_splits = n_splits, shuffle = True, random_state = 7777)

In [114]:
for train_idx, valid_idx in folds.split(train):
    train_f, valid_f = train.iloc[train_idx], train.iloc[valid_idx]
    train_f, valid_f = map(lambda df: df.reset_index(drop = True), (train_f, valid_f))
    train_gen, valid_gen = map(lambda x: make_generator(x, n_item, batch_size, implicit = True, user_specific = False), (train_f, valid_f))
    steps_per_epoch = train_f.index.size // batch_size + 1
    validation_steps = valid_f.index.size // batch_size + 1

    early_stopping = EarlyStopping(monitor = 'val_loss', mode = 'min', verbose = 1, patience = 50)
    model_checkpoint = ModelCheckpoint('DAE.h5', monitor = 'val_loss', mode = 'min', save_best_only = True)

    hist = model.fit(x = train_gen, validation_data = valid_gen, epochs = epochs,
                     steps_per_epoch = steps_per_epoch, validation_steps = validation_steps,
                     verbose = 1, callbacks = [early_stopping, model_checkpoint])

Epoch 1/100
38/38 [==============================] - 1s 33ms/step - loss: 1.1498 - masked_cross_entropy: 0.4586 - masked_rmse: 0.3693 - val_loss: 0.7896 - val_masked_cross_entropy: 0.3237 - val_masked_rmse: 0.2949
Epoch 2/100
38/38 [==============================] - 1s 34ms/step - loss: 0.6317 - masked_cross_entropy: 0.3072 - masked_rmse: 0.2888 - val_loss: 0.5105 - val_masked_cross_entropy: 0.2969 - val_masked_rmse: 0.2840
Epoch 3/100
38/38 [==============================] - 1s 34ms/step - loss: 0.4402 - masked_cross_entropy: 0.2778 - masked_rmse: 0.2743 - val_loss: 0.3895 - val_masked_cross_entropy: 0.2661 - val_masked_rmse: 0.2683
Epoch 4/100
38/38 [==============================] - 1s 32ms/step - loss: 0.3595 - masked_cross_entropy: 0.2564 - masked_rmse: 0.2639 - val_loss: 0.3375 - val_masked_cross_entropy: 0.2510 - val_masked_rmse: 0.2609
Epoch 5/100
38/38 [==============================] - 1s 29ms/step - loss: 0.3218 - masked_cross_entropy: 0.2440 - masked_rmse: 0.2575 - val_loss

In [136]:
# show_history(hist, 'loss', 'masked_cross_entropy')

In [116]:
def ndcg(gt, rec):
    dcg, idcg = 0.0, 0.0
    for i, r in enumerate(rec):
        idcg += 1.0 / np.log(i + 2)
        if r in gt:
            dcg += 1.0 / np.log(i + 2)
    return dcg / idcg

def recall(gt, rec):
    res = [r for r in rec if r in gt]
    return len(res) / len(rec)

def precision(gt, rec):
    res = [r for r in rec if r in gt]
    return len(res) / len(gt)

In [117]:
 pred = make_predictor(train, model, n_item, batch_size, user_specific = False)

In [118]:
m_ndcg = 0
m_recall = 0
m_precision = 0
for i in range(len(pred)):
    rec = pred[i]
    gt = test.at[i, 'movieId']
    m_ndcg += ndcg(gt, rec)
    m_recall += recall(gt, rec)
    m_precision += precision(gt, rec)
m_ndcg /= len(pred)
m_recall /= len(pred)
m_precision /= len(pred)

In [119]:
m_ndcg

0.20286943293069754

In [120]:
m_recall

0.17192575406032365

In [121]:
m_precision

0.22572257081982663

## 3. CDAE

In [123]:
optimizer = Adam(learning_rate = 0.001, decay = 1e-5)
model = CDAE(n_user, n_item, latent_dim = 512, optimizer = optimizer, user_specific = True)
model.summary()

Model: "functional_7"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
user (InputLayer)               [(None, 1)]          0                                            
__________________________________________________________________________________________________
input (InputLayer)              [(None, 3706)]       0                                            
__________________________________________________________________________________________________
embedding (Embedding)           (None, 1, 512)       3092480     user[0][0]                       
__________________________________________________________________________________________________
encoder (Dense)                 (None, 512)          1897984     input[0][0]                      
_______________________________________________________________________________________

In [124]:
train, test = data_split(ratings)
train, test = map(make_interaction, (train, test))

In [125]:
epochs = 100
batch_size = 128

n_splits = 5
folds = KFold(n_splits = n_splits, shuffle = True, random_state = 7777)

In [126]:
for train_idx, valid_idx in folds.split(train):
    train_f, valid_f = train.iloc[train_idx], train.iloc[valid_idx]
    train_f, valid_f = map(lambda df: df.reset_index(drop = True), (train_f, valid_f))
    train_gen, valid_gen = map(lambda x: make_generator(x, n_item, batch_size, implicit = True, user_specific = True), (train_f, valid_f))
    steps_per_epoch = train_f.index.size // batch_size + 1
    validation_steps = valid_f.index.size // batch_size + 1

    early_stopping = EarlyStopping(monitor = 'val_loss', mode = 'min', verbose = 1, patience = 50)
    model_checkpoint = ModelCheckpoint('DAE.h5', monitor = 'val_loss', mode = 'min', save_best_only = True)

    hist = model.fit(x = train_gen, validation_data = valid_gen, epochs = epochs,
                     steps_per_epoch = steps_per_epoch, validation_steps = validation_steps,
                     verbose = 1, callbacks = [early_stopping, model_checkpoint])

Epoch 1/100


/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/indexed_slices.py:432: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


38/38 [==============================] - 2s 53ms/step - loss: 1.5521 - masked_cross_entropy: 0.4643 - masked_rmse: 0.3722 - val_loss: 0.9552 - val_masked_cross_entropy: 0.3280 - val_masked_rmse: 0.2976
Epoch 2/100
38/38 [==============================] - 3s 81ms/step - loss: 0.7205 - masked_cross_entropy: 0.3076 - masked_rmse: 0.2890 - val_loss: 0.5505 - val_masked_cross_entropy: 0.2948 - val_masked_rmse: 0.2817
Epoch 3/100
38/38 [==============================] - 2s 49ms/step - loss: 0.4650 - masked_cross_entropy: 0.2781 - masked_rmse: 0.2745 - val_loss: 0.4016 - val_masked_cross_entropy: 0.2661 - val_masked_rmse: 0.2684
Epoch 4/100
38/38 [==============================] - 3s 84ms/step - loss: 0.3667 - masked_cross_entropy: 0.2560 - masked_rmse: 0.2635 - val_loss: 0.3424 - val_masked_cross_entropy: 0.2518 - val_masked_rmse: 0.2617
Epoch 5/100
38/38 [==============================] - 2s 50ms/step - loss: 0.3238 - masked_cross_entropy: 0.2439 - masked_rmse: 0.2576 - val_loss: 0.3133 - v

In [137]:
# show_history(hist, 'loss', 'masked_cross_entropy')

In [128]:
def ndcg(gt, rec):
    dcg, idcg = 0.0, 0.0
    for i, r in enumerate(rec):
        idcg += 1.0 / np.log(i + 2)
        if r in gt:
            dcg += 1.0 / np.log(i + 2)
    return dcg / idcg

def recall(gt, rec):
    res = [r for r in rec if r in gt]
    return len(res) / len(rec)

def precision(gt, rec):
    res = [r for r in rec if r in gt]
    return len(res) / len(gt)

In [131]:
 pred = make_predictor(train, model, n_item, batch_size, user_specific = True)

In [132]:
m_ndcg = 0
m_recall = 0
m_precision = 0
for i in range(len(pred)):
    rec = pred[i]
    gt = test.at[i, 'movieId']
    m_ndcg += ndcg(gt, rec)
    m_recall += recall(gt, rec)
    m_precision += precision(gt, rec)
m_ndcg /= len(pred)
m_recall /= len(pred)
m_precision /= len(pred)

In [133]:
m_ndcg

0.20215455440771032

In [134]:
m_recall

0.17193404043751914

In [135]:
m_precision

0.2255756595622609